<a href="https://colab.research.google.com/github/luisanovaes/GitHub/blob/master/Classificador_BERT_Portugues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 5.3 MB/s 
     |████████████████████████████████| 2.8 MB 42.2 MB/s 
     |████████████████████████████████| 43 kB 1.3 MB/s 
     |████████████████████████████████| 1.2 MB 35.5 MB/s 
     |████████████████████████████████| 270 kB 53.2 MB/s 
     |████████████████████████████████| 124 kB 60.8 MB/s 
     |████████████████████████████████| 8.0 MB 6.6 MB/s 
     |████████████████████████████████| 3.3 MB 42.3 MB/s 
     |████████████████████████████████| 1.7 MB 31.6 MB/s 
     |████████████████████████████████| 636 kB 33.5 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 895 kB 31.9 MB/s 
     |████████████████████████████████| 133 kB 47.2 MB/s 
     |████████████████████████████████| 180 kB 45.5 MB/s 
     |████████████████████████████████| 97 kB 5.2 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 119 kB 46.6 MB/s 
     |██████████████████

In [1]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

##importando a tabela
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1eS2a9h676hpFhQsisSdJCgdqYaDpReyMgL-eREZ4-XQ/edit#gid=1423440435')

#Selecionando os dados de cada aba
sheet1 = wb.worksheet('Treino')
sheet2 = wb.worksheet('Teste')

#Selecionando os dados e passando para um dataframe
data1 = sheet1.get_all_values()
data2 = sheet2.get_all_values()

train_df = pd.DataFrame(data1)
train_df.columns = ['query','classificacao']
train_df['classificacao'] = train_df['classificacao'].astype(int)


test_df = pd.DataFrame(data2)
test_df.columns = ['query','classificacao']
test_df['classificacao'] = test_df['classificacao'].astype(int)


#vizualizar dados
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   query          361 non-null    object
 1   classificacao  361 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.8+ KB


In [3]:
# Classificador

from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
#train_data = train_df

# Preparing eval data
#eval_data = test_df


# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1)
model_args.overwrite_output_dir = True

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'neuralmind/bert-base-portuguese-cased',
    num_labels=3,
    use_cuda=False,
    args=model_args,   
) 

# Train the model
model.train_model(train_df)
model_args.overwrite_output_dir =True


# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)




INFO:filelock:Lock 139912293752720 acquired on /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716.lock


Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

INFO:filelock:Lock 139912293752720 released on /root/.cache/huggingface/transformers/e716e2151985ba669e7197b64cdde2552acee146494d40ffaf0688a3f152e6ed.18a0b8b86f3ebd4c8a1d8d6199178feae9971ff5420f1d12f0ed8326ffdff716.lock
INFO:filelock:Lock 139912346767888 acquired on /root/.cache/huggingface/transformers/1e42c907c340c902923496246dae63e33f64955c529720991b7ec5543a98e442.fa492fca6dcee85bef053cc60912a211feb1f7173129e4eb1a5164e817f2f5f2.lock


Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

INFO:filelock:Lock 139912346767888 released on /root/.cache/huggingface/transformers/1e42c907c340c902923496246dae63e33f64955c529720991b7ec5543a98e442.fa492fca6dcee85bef053cc60912a211feb1f7173129e4eb1a5164e817f2f5f2.lock
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of 

...

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1402: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/361 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_3_2


Running Evaluation:   0%|          | 0/46 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.7851097091498678, 'eval_loss': 0.26576594970917894}


In [5]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["Petição Inicial","lei pelé","art. 3 da cf", "06234504531"])

print(result,'\n')
print(model_outputs, '\n')
print(predictions, '\n', raw_outputs)
print(wrong_predictions)


df_output = pd.DataFrame(model_outputs)
df_wrong_predictions = pd.DataFrame(wrong_predictions)

df_output.to_csv('drive/MyDrive/Results-BERT/output.csv')




INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'mcc': 0.7851097091498678, 'eval_loss': 0.26576594970917894} 

[[-2.03492546  3.58493495 -1.56247938]
 [-2.02145123  3.43498135 -1.43497753]
 [-2.12955952  3.74755621 -1.6248982 ]
 ...
 [ 1.22697854 -2.54768562  1.52557552]
 [ 1.42584372 -3.43310118  1.97793031]
 [ 2.31462002 -2.35470653  0.11816008]] 

[2 1 1 0] 
 [[-0.02047378 -3.13972402  2.90651131]
 [-2.03004408  3.71315742 -1.70804524]
 [-1.58917904  2.50021839 -1.10977781]
 [ 2.57293892 -1.74438    -0.42247739]]
[]
